##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo crear una canalización de TFX usando plantillas con el orquestador local

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td>     <a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

## Introducción

Este documento proporcionará instrucciones para crear una canalización de TensorFlow Extended (TFX) utilizando las *plantillas* que se proporcionan con el paquete de Python para TFX. La mayoría de las instrucciones son comandos de shell de Linux y se proporcionan las correspondientes celdas de código de Jupyter Notebook que invocan esos comandos usando `!`.

Compilará una canalización mediante el uso del [conjunto de datos Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) publicado por la ciudad de Chicago. Le recomendamos que use esta canalización como punto de referencia para intentar compilar su propia canalización con su conjunto de datos.

Compilaremos una canalización que funcione en el entorno local. Si le interesa usar el orquestador de Kubeflow en Google Cloud, consulte el [tutorial de TFX en Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

## Requisitos previos

- Linux / MacOS
- Python &gt;= 3.5.3

Puede obtener todos los requisitos previos fácilmente [mediante la ejecución de este bloc de notas en Google Colab](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb).


## Paso 1. Configure su entorno

**A lo largo de este documento, presentaremos comandos dos veces. Una vez como comando de shell listo para copiar y pegar, otra vez como celda de bloc de notas Jupyter. Si está usando Colab, simplemente omita el bloque del script de shell y ejecute las celdas del bloc de notas.**

Debe preparar un entorno de desarrollo para compilar una canalización.

Instale el paquete de python par a`tfx`. Recomendamos el uso de `virtualenv` en el entorno local. Puede utilizar el siguiente fragmento de script de Shell para configurar su entorno.

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

Si está usando Colab:


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

NOTA: Es posible que se produzcan algunos errores durante la instalación del paquete. Por ejemplo,

> ERROR: algún-paquete 0.alguna_versión.1 tiene como requisito ¡otro-paquete!=2.0.,&lt;3,&gt;=1.15, pero tendrá otro-paquete 2.0.0 que es incompatible.

Ignore estos errores en este momento.

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

Comprobemos la versión de TFX.

```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

Y ya está. Estamos listos para crear una canalización.

## Paso 2. Copie la plantilla predefinida al directorio de su proyecto

En este paso, crearemos un directorio y archivos de proyecto de canalización de trabajo copiando archivos adicionales de una plantilla predefinida.

Puede cambiar el nombre de su canalización si cambia el valor de `PIPELINE_NAME` a continuación. Este también se convertirá en el nombre del directorio del proyecto donde se colocarán sus archivos.

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX incluye la plantilla `taxi` con el paquete de Python para TFX. Si planea resolver un problema de predicción puntual, que incluye clasificación y regresión, esta plantilla podría usarse como punto de partida.

El comando de CLI `tfx template copy` copia archivos de plantilla predefinidos al directorio de su proyecto.

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

Cambie el contexto del directorio de trabajo en este bloc de notas al directorio del proyecto.

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## Paso 3. Exploración de archivos fuente copiados

La plantilla de TFX proporciona archivos de estructura básicos para compilar una canalización, incluido el código fuente de Python, datos de muestra y bloc de notas Jupyter para analizar la salida de la canalización. La plantilla `taxi` utiliza el mismo conjunto de datos *Chicago Taxi* y el mismo modelo de ML que el [Tutorial de Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

En Google Colab, puede buscar archivos al hacer clic en el icono de una carpeta a la izquierda. Los archivos deben copiarse en el directorio del proyecto, cuyo nombre es `my_pipeline` en este caso. Puede hacer clic en los nombres de los directorios para ver el contenido del directorio y hacer doble clic en los nombres de los archivos para abrirlos.

Esta es una breve introducción a cada uno de los archivos de Python.

- `pipeline`: este directorio contiene la definición de la canalización.
    - `configs.py`: define constantes comunes para los ejecutores de la canalización
    - `pipeline.py`: define los componentes de TFX y una canalización
- `models`: este directorio contiene definiciones de modelos de ML.
    - `features.py`, `features_test.py`: define características para el modelo
    - `preprocessing.py`, `preprocessing_test.py`: define trabajos de preprocesamiento con ayuda de `tf::Transform`
    - `estimator`: este directorio contiene un modelo basado en Estimator.
        - `constants.py`: define las constantes del modelo
        - `model.py`, `model_test.py`: define el modelo DNN utilizando el estimador TF
    - `keras`: este directorio contiene un modelo basado en Keras.
        - `constants.py`: define las constantes del modelo
        - `model.py`, `model_test.py`: define el modelo DNN usando Keras
- `local_runner.py`, `kubeflow_runner.py`: define ejecutores para cada motor de orquestación


Quizás note que hay algunos archivos con `_test.py` en su nombre. Estas son pruebas unitarias de la canalización y se recomienda agregar más pruebas unitarias a medida que implemente sus propias canalizaciones. Puede ejecutar pruebas unitarias si proporciona el nombre del módulo de los archivos de prueba con la marca `-m`. Generalmente puede obtener el nombre de un módulo al eliminar la extensión `.py` y reemplazarla por `/` con `.`. Por ejemplo:

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## Paso 4. Ejecute su primera canalización de TFX

Puede crear una canalización a partir del comando `pipeline create`.

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

Luego, puede ejecutar la canalización creada a partir del comando `run create`.

```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Si tiene éxito, verá que `Component CsvExampleGen is finished.` Cuando copia la plantilla, solo se incluye un componente, CsvExampleGen, en la canalización.

## Paso 5. Agregue componentes para la validación de datos

En este paso, agregará componentes para la validación de datos, incluidos `StatisticsGen`, `SchemaGen` y `ExampleValidator`. Si está interesado en la validación de datos, consulte [Introducción a Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

Modificaremos la definición de canalización copiada en `pipeline/pipeline.py`. Si está trabajando en su entorno local, utilice su editor favorito para editar el archivo. Si está trabajando en Google Colab,

> **Haga clic en el icono de carpeta a la izquierda para abrir la vista `Files`**.

> **Haga clic en `my_pipeline` para abrir el directorio y haga clic en el directorio `pipeline` para abrir y haga doble clic en `pipeline.py` para abrir el archivo**.

> Busque y descomente las 3 líneas que agregan `StatisticsGen`, `SchemaGen` y `ExampleValidator` a la canalización. (Consejo: busque comentarios que contengan `TODO(step 5):` :).

> Su cambio se guardará automáticamente en unos segundos. Asegúrese de que la marca `*` delante de `pipeline.py` desaparezca en el título de la pestaña. **No hay ningún botón para guardar ni acceso directo para el editor de archivos en Colab. Los archivos Python en el editor de archivos se pueden guardar en el entorno de ejecución incluso en el modo `playground`.**

Ahora debe actualizar la canalización existente con la definición de canalización modificada. Utilice el comando `tfx pipeline update` para actualizar su canalización, seguido del comando `tfx run create` para crear una nueva ejecución de su canalización actualizada.

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

Debería poder ver el registro de salida de los componentes agregados. Nuestra canalización crea artefactos de salida en el directorio `tfx_pipeline_output/my_pipeline`.

## Paso 6. Agregue componentes para el entrenamiento

En este paso, agregará componentes para el entrenamiento y la validación del modelo, incluidos `Transform`, `Trainer`, `Resolver`, `Evaluator` y `Pusher`.

> **Abra `pipeline/pipeline.py`**. Busque y descomente 5 líneas que agregan `Transform`, `Trainer`, `Resolver`, `Evaluator` y `Pusher` a la canalización. (Consejo: busque `TODO(step 6):`

Como hizo antes, ahora necesita actualizar la canalización existente con la definición de canalización modificada. Las instrucciones son las mismas que las del Paso 5. Actualice la canalización con ayuda de `tfx pipeline update` y use `tfx run create` para crear una ejecución.

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

Cuando esta ejecución finalice correctamente, habrá creado y ejecutado su primera canalización de TFX con el orquestador local.

**NOTA:** Es posible que haya notado que cada vez que creamos una ejecución de canalización, cada componente se ejecuta una y otra vez aunque la entrada y los parámetros no hayan cambiado. Esto implica una pérdida de tiempo y recursos, y puede omitir esas ejecuciones con el almacenamiento en caché de la canalización. Puede habilitar el almacenamiento en caché si especifica `enable_cache=True` para el objeto `Pipeline` en `pipeline.py`.


## Paso 7. (*Opcional*) Pruebe BigQueryExampleGen

<a>BigQuery</a> es un almacén de datos en la nube sin servidor, altamente escalable y rentable. BigQuery sirve como fuente de ejemplos de entrenamiento en TFX. En este paso, agregaremos <code>BigQueryExampleGen</code> a la canalización.

Necesita una cuenta [de Google Cloud Platform](https://cloud.google.com/gcp/getting-started) para usar BigQuery. Prepare un proyecto de GCP.

Use la biblioteca de autenticación de colab o la utilidad `gcloud` para iniciar sesión en su proyecto.

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


Debe especificar el nombre de su proyecto de GCP para acceder a los recursos de BigQuery mediante TFX. Establezca la variable de entorno `GOOGLE_CLOUD_PROJECT` en el nombre de su proyecto.

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **Abra `pipeline/pipeline.py`**. Comente `CsvExampleGen` y descomente la línea que crea una instancia de `BigQueryExampleGen`. También debe descomentar el argumento `query` de la función `create_pipeline`.

Tenemos que especificar nuevamente qué proyecto de GCP se usará para BigQuery, y para hacer esto hay que configurar `--project` en `beam_pipeline_args` cuando se crea una canalización.

> **Abra `pipeline/configs.py`**. Descomente la definición de `BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` y `BIG_QUERY_QUERY`. Debe reemplazar la identificación del proyecto y el valor de la región en este archivo con los valores correctos para su proyecto de GCP.

> **Abra `local_runner.py`**. Descomente dos argumentos, `query` y `beam_pipeline_args`, para el método create_pipeline().

Ahora la canalización está lista para usar BigQuery como fuente de ejemplo. Actualice la canalización y cree una ejecución como lo hicimos en los pasos 5 y 6.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## Siguientes pasos: ingiera SUS datos en la canalización

Creamos una canalización para un modelo utilizando el conjunto de datos Chicago Taxi. Ahora es el momento de poner sus datos en proceso.

Sus datos se pueden almacenar en cualquier lugar al que pueda acceder su canalización, incluido GCS o BigQuery. Deberá modificar la definición de la canalización para acceder a sus datos.

1. Si sus datos están almacenados en archivos, modifique `DATA_PATH` en `kubeflow_runner.py` o `local_runner.py` y configúrelo en la ubicación de sus archivos. Si sus datos están almacenados en BigQuery, modifique `BIG_QUERY_QUERY` en `pipeline/configs.py` para consultar correctamente sus datos.
2. Agregue características en `models` / <code>features.py</code>.
3. Modifique `models` / <code>preprocessing.py</code> para [transformar los datos de entrada para el entrenamiento](https://www.tensorflow.org/tfx/guide/transform).
4. Modifique `models/keras/model.py` y `models/keras/constants.py` para [describir su modelo de ML](https://www.tensorflow.org/tfx/guide/trainer).
    - También puede utilizar un modelo basado en un estimador. Cambie la constante `RUN_FN` a `models.estimator.model.run_fn` en `pipeline/configs.py`.

Consulte la [guía del componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para obtener más información.